<a href="https://colab.research.google.com/github/silverstar0727/kfp-tutorial/blob/main/tfx-pipeline-tutorial/penguin_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Written by TensorFlow

Translated & Modified by Jeongmin-Do

In [ ]:
!pip install --upgrade pip

     |████████████████████████████████| 1.6MB 6.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


### Install TFX


In [ ]:
# 실행 후 런타임 다시시작을 해주세요
!pip install -q tfx

distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 2.3 MB 6.5 MB/s 
     |████████████████████████████████| 1.5 MB 28.6 MB/s 
     |████████████████████████████████| 400 kB 41.0 MB/s 
     |████████████████████████████████| 9.6 MB 33.2 MB/s 
     |████████████████████████████████| 89 kB 2.5 MB/s 
     |████████████████████████████████| 147 kB 65.1 MB/s 
     |████████████████████████████████| 1.2 MB 69.7 MB/s 
     |████████████████████████████████| 1.7 MB 73.1 MB/s 
     |████████████████████████████████| 1.4 MB 60.5 MB/s 
     |████████████████████████████████| 2.2 MB 66.3 MB/s 
     |███████████

### Set up variables

In [ ]:
import os

PIPELINE_NAME = "penguin-simple"

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)                        # 아티팩트가 저장될 경로
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')          # ML metadata 저장소로 SQLite DB 사용경로
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)                # 모델이 서빙될 경로

from absl import logging
logging.set_verbosity(logging.INFO)

### Prepare example data
[Palmer Penguins dataset](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)

데이터셋 features
- culmen_length_mm : 부리 길이
- culmen_depth_mm : 부리 깊이
- flipper_length_mm : 날개 길이
- body_mass_g : 몸무게

이미 모든 feature는 normalized가 되어있고, 종을 예측하는 분류문제이다.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

('/tmp/tfx-datadeiuxyl1/data.csv', <http.client.HTTPMessage at 0x7f68d7e0c790>)

In [ ]:
# 파일 일부 확인
!head {_data_filepath}

species,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g
0,0.2545454545454545,0.6666666666666666,0.15254237288135594,0.2916666666666667
0,0.26909090909090905,0.5119047619047618,0.23728813559322035,0.3055555555555556
0,0.29818181818181805,0.5833333333333334,0.3898305084745763,0.1527777777777778
0,0.16727272727272732,0.7380952380952381,0.3559322033898305,0.20833333333333334
0,0.26181818181818167,0.892857142857143,0.3050847457627119,0.2638888888888889
0,0.24727272727272717,0.5595238095238096,0.15254237288135594,0.2569444444444444
0,0.25818181818181823,0.773809523809524,0.3898305084745763,0.5486111111111112
0,0.32727272727272727,0.5357142857142859,0.1694915254237288,0.1388888888888889
0,0.23636363636363636,0.9642857142857142,0.3220338983050847,0.3055555555555556


You should be able to see five values. `species` is one of 0, 1 or 2, and all
other features should have values between 0 and 1.

## Create a pipeline

- CsvExampleGen: data file을 TFRecord 파일로 변환.
- Trainer: 모델을 별도의 파이썬 파일로 작성하고 이를 이용하여 Train 컴포넌트로 훈련을 진행.
- Pusher: 훈련된 ML model을 파이프라인 외부로 배포하는 과정을 거침.


### Write model training code

Trainer 컴포넌트를 실행하기 위한 훈련 스크립트 작성. 해당 파이썬 스크립트는 run_fn 함수를 반드시 포함하고 있어야 하고, Trainer는 이 함수를 실행하는 방식으로 진행.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAccessor
from tfx_bsl.tfxio import dataset_options
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  return data_accessor.tf_dataset_factory(
      file_pattern,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)
  
  model.save(fn_args.serving_model_dir, save_format='tf')

Writing penguin_trainer.py


### Write a pipeline definition

아래의 세 컴포넌트를 포함하는 파이프라인 함수 작성 
* ExampleGen
* Trainer
* Pusher

In [ ]:
from tfx.components import CsvExampleGen
from tfx.components import Pusher
from tfx.components import Trainer
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.components.base import executor_spec
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2

# pipeline.Pipeline을 반환하는 파이프라인 함수 작성
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> pipeline.Pipeline:
  # ExampleGen
  example_gen = CsvExampleGen(input_base=data_root)                             # input data

  # Trainer
  trainer = Trainer(
      module_file=module_file,                                                  # 앞서 생성한 훈련 스크립트
      custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),    # GenericExecutor를 사용한 로컬 훈련
      examples=example_gen.outputs['examples'],                                 # input data(=Examples)
      train_args=trainer_pb2.TrainArgs(num_steps=100),                          # train 인자
      eval_args=trainer_pb2.EvalArgs(num_steps=5))                              # evaluation 인자

  # Pusher
  pusher = Pusher(
      model=trainer.outputs['model'],                                           # trainer에서의 훈련된 모델을 사용
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=serving_model_dir)))                               # "serving_model/penguin-simple"로 모델 서빙

  # 파이프라인에 위 세 컴포넌트를 포함.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return pipeline.Pipeline(
      pipeline_name=pipeline_name,                                              # pipeline 이름 지정
      pipeline_root=pipeline_root,                                              # pipeline 저장 경로 지정
      metadata_connection_config=metadata.sqlite_metadata_connection_config(    # metadata 연결 구성
          metadata_path),
      components=components)                                                    # 컴포넌트 지정

INFO:absl:tensorflow_ranking is not available: No module named 'tensorflow_ranking'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'
INFO:absl:tensorflow_text is not available: No module named 'tensorflow_text'


## Run the pipeline

TFX의 local_dag_runner를 이용한 오케스트레이터 실행

오케스트레이터로는 다음의 세 가지가 가능
* Kubeflow
* Local
* Apache Airflow

In [ ]:
import os
from tfx.orchestration.local import local_dag_runner

local_dag_runner.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

파이프라인이 성공적으로 완료되면 로그 끝에 "INFO:absl:Component Pusher is finished."가 표시됨.

Pusher를 통해 SERVING_MODEL_DIR로 서빙. Colab의 왼쪽 패널에 있는 파일 탐색기 또는 다음 명령을 사용하여 결과를 확인.

In [ ]:
!find {SERVING_MODEL_DIR}

serving_model/penguin-simple
serving_model/penguin-simple/1619597461
serving_model/penguin-simple/1619597461/assets
serving_model/penguin-simple/1619597461/variables
serving_model/penguin-simple/1619597461/variables/variables.data-00000-of-00001
serving_model/penguin-simple/1619597461/variables/variables.index
serving_model/penguin-simple/1619597461/saved_model.pb
